<a href="https://colab.research.google.com/github/JayantGoel001/GraphsAndSocialNetworkAnalytics/blob/master/Project/Link_Prediction_on_Facebook_Recruiting_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /drive/MyContent/train.csv .
!cp /drive/MyContent/test.csv .

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
df = pd.read_csv('train.csv',index_col=False)
df.head()

,source_node,destination_node
0,1,690569
1,1,315892
2,1,189226
3,2,834328
4,2,1615927


In [ ]:
edge_list = df.values.tolist()

In [ ]:
G = nx.DiGraph(edge_list)

In [ ]:
nx.info(G)

'DiGraph with 1862220 nodes and 9437519 edges'